In [1]:
# 데이터 불러오기 및 기본 정보 확인
import pandas as pd

# 파일 경로
file_path = "datasets/BostonHousingprice.csv"

# 데이터 읽기
data = pd.read_csv(file_path)

# 데이터 확인
print(data.head())
print(data.info())
print(data.describe())

      CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
0  0.00632  18.0   2.31     0  0.538  6.575  65.2  4.0900    1  296.0   
1  0.02731   0.0   7.07     0  0.469  6.421  78.9  4.9671    2  242.0   
2  0.02729   0.0   7.07     0  0.469  7.185  61.1  4.9671    2  242.0   
3  0.03237   0.0   2.18     0  0.458  6.998  45.8  6.0622    3  222.0   
4  0.06905   0.0   2.18     0  0.458  7.147  54.2  6.0622    3  222.0   

   PTRATIO       B  LSTAT  MEDV  
0     15.3  396.90   4.98  24.0  
1     17.8  396.90   9.14  21.6  
2     17.8  392.83   4.03  34.7  
3     18.7  394.63   2.94  33.4  
4     18.7  396.90   5.33  36.2  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2024 entries, 0 to 2023
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     2024 non-null   float64
 1   ZN       2024 non-null   float64
 2   INDUS    2024 non-null   float64
 3   CHAS     2024 non-null   int64  
 4   NOX      2024 non

In [9]:
# 결측치 확인
print(data.isnull().sum())

# 이상치 탐지를 위한 간단한 Boxplot
import matplotlib.pyplot as plt
data['price'].boxplot()
plt.title("Price Boxplot")
plt.show()


CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         5
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
MEDV       0
dtype: int64


KeyError: 'price'

In [ ]:
# 상관관계 분석 및 히트맵
import seaborn as sns
import matplotlib.pyplot as plt

correlation_matrix = data.corr()

# 히트맵 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', cbar=True)
plt.title("Feature Correlation Heatmap")
plt.show()


In [ ]:
# 변수별 시각화 (Scatterplot)
sns.scatterplot(x='CRIM', y='price', data=data)
plt.title("CRIM vs Price")
plt.xlabel("Crime Rate (CRIM)")
plt.ylabel("House Price (Price)")
plt.show()

sns.scatterplot(x='RM', y='price', data=data)
plt.title("RM vs Price")
plt.xlabel("Number of Rooms (RM)")
plt.ylabel("House Price (Price)")
plt.show()

sns.scatterplot(x='LSTAT', y='price', data=data)
plt.title("LSTAT vs Price")
plt.xlabel("% Lower Status of Population (LSTAT)")
plt.ylabel("House Price (Price)")
plt.show()


In [ ]:
# 데이터 분포 확인 (히스토그램)
data['price'].hist(bins=30, edgecolor='black')
plt.title("Price Distribution")
plt.xlabel("House Price")
plt.ylabel("Frequency")
plt.show()

data['RM'].hist(bins=30, edgecolor='black')
plt.title("RM Distribution")
plt.xlabel("Number of Rooms (RM)")
plt.ylabel("Frequency")
plt.show()


보스턴 집값 예측